In [2]:
import pandas as pd
from pymongo import MongoClient
client = MongoClient()
db = client.mydisease.disgenet

In [3]:
url = "http://www.disgenet.org/ds/DisGeNET/results/curated_gene_disease_associations.tsv.gz"
df = pd.read_csv(url, sep='\t', comment='#', compression='gzip')

In [4]:
rename = {'diseaseId': '_id',
          'geneId': 'gene_id',
          'geneName': 'gene_name',
          'diseaseName': 'label',
          'sourceId': 'source',
          'NofPmids': '#pmids',
          'NofSnps': '#snps'}

In [5]:
df.sourceId = df.sourceId.str.split(",")

In [7]:
df = df.rename(columns=rename)

In [8]:
df.head()

,_id,gene_id,score,gene_name,description,label,source,#pmids,#snps
0,umls:C0000737,3440,0.120000,IFNA2,"interferon, alpha 2",Abdominal Pain,[CTD_human],1,0
1,umls:C0000744,4547,0.490868,MTTP,microsomal triglyceride transfer protein,Abetalipoproteinemia,"[CLINVAR, CTD_human, ORPHANET, UNIPROT]",33,5
2,umls:C0000768,10683,0.121086,DLL3,delta-like 3 (Drosophila),Congenital Abnormality,[CTD_human],4,0
3,umls:C0000768,1295,0.120000,COL8A1,"collagen, type VIII, alpha 1",Congenital Abnormality,[CTD_human],1,0
4,umls:C0000768,1317,0.120000,SLC31A1,"solute carrier family 31 (copper transporter),...",Congenital Abnormality,[CTD_human],1,0


In [15]:
d = []
for did, subdf in df.groupby("_id"):
    records = subdf.to_dict(orient='records')
    records = [{k:v for k,v in record.items() if k not in {'_id','label', 'description'}} for record in records]
    drecord = {'_id': did.replace("umls","umls_cui"), 'genes': records}
    d.append(drecord)
print(len(d))

7607


In [20]:
d[10]

{'_id': 'umls_cui:C0001126',
 'genes': [{'#pmids': 12,
   '#snps': 1,
   'gene_id': 6521,
   'gene_name': 'SLC4A1',
   'score': 0.142337685683898,
   'source': ['CTD_human']},
  {'#pmids': 3,
   '#snps': 0,
   'gene_id': 8671,
   'gene_name': 'SLC4A4',
   'score': 0.12817305072740198,
   'source': ['CTD_human']}]}

In [17]:
db.drop()
db.insert_many(d)

In [21]:
db.find_one('umls_cui:C0001126')

{'_id': 'umls_cui:C0001126',
 'genes': [{'#pmids': 12,
   '#snps': 1,
   'gene_id': 6521,
   'gene_name': 'SLC4A1',
   'score': 0.142337685683898,
   'source': ['CTD_human']},
  {'#pmids': 3,
   '#snps': 0,
   'gene_id': 8671,
   'gene_name': 'SLC4A4',
   'score': 0.12817305072740198,
   'source': ['CTD_human']}]}

In [22]:
jsonld = {
    "disgenet": {
        "@context": {"genes": ""}
    },
    "disgenet/genes": {
        "@context": {
            "gene_id": "http://identifiers.org/ncbigene/",
            "gene_name": "http://identifiers.org/orphanet.ordo/"
        }
    }
}